In [1]:
# pour la manipulation et l'analyse des donnees
import pandas as pd

# pour les calculs numeriques
import numpy as np

# pour ouvrir les URL
from urllib.request import urlopen
# pour extraire le contenu HTML
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin, urlparse, quote

import sqlite3, os

import time

In [2]:
!pip install pymongo

In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/", 
                     serverSelectionTimeoutMS=5000, 
                     connectTimeoutMS=10000,
                     socketTimeoutMS=None, # Pas de limite sur le temps de reponse
                     retryWrites=True)
db = client["SmartSearch"]      # nom de la bd
produits_col = db["produits"]

In [15]:
# on evite les doublons 

produits_col.create_index("url", unique=True)

'url_1'

In [ ]:
# on vide
"""nb_suppr = produits_col.delete_many({}) # {} pour tout selectionner
print(f"Nettoyage : {nb_suppr.deleted_count} produits supprimés de MongoDB.")

produits_col.create_index("url", unique=True)"""

Nettoyage : 976 produits supprimés de MongoDB.


'url_1'

In [18]:
DOSSIER_IMG = 'ImagesTech'
if not os.path.exists(DOSSIER_IMG):
    os.makedirs(DOSSIER_IMG)

In [19]:
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}

In [20]:
url_base = "https://nowtechcenter.com/boutique/"

In [ ]:
page = 1
while True:
    url_page = f"{url_base}page/{page}/"
    print(f"\n==> Page {page} --> {url_page}")

    try:
        response = requests.get(url_page, headers=HEADERS, timeout=15)
        #if response.status_code != 200: break

        if response.status_code not in [200, 301, 302]: 
            print(f"Erreur d'accès : {response.status_code}")
            break
    except: break

    soup = BeautifulSoup(response.text, "html.parser")
    links = list(dict.fromkeys([a.get("href") for a in soup.select("a.woocommerce-LoopProduct-link")]))

    print(f"Nombre de produits trouvés sur cette page : {len(links)}") 

    if not links:
        print("Attention : Aucun lien trouvé. Le sélecteur CSS est peut-être incorrect.")
        
    for product_url in links:
        # on verifie les doublons
        if produits_col.find_one({"url": product_url}):
            print(f"Déjà en base : {product_url}")
            continue

        print(f"Extraction : {product_url}")
        try:
            prod_resp = requests.get(product_url, headers=HEADERS, timeout=15)
            ps = BeautifulSoup(prod_resp.text, "html.parser")

            # Extraction 
            nom = ps.find('h1').get_text(strip=True) if ps.find('h1') else "Inconnu"
            prix_avant_reduction = ps.find("del").find("bdi").get_text(strip=True) if ps.find("del") else None

            # prix actuel
            p_ins = ps.find("ins")
            if p_ins:
                prix_apres_reduction = p_ins.find("bdi").get_text(strip=True)
            else:
                p_tag = ps.find("p", class_="price")
                prix_apres_reduction = p_tag.find("bdi").get_text(strip=True) if p_tag and p_tag.find("bdi") else None

            reduction = ps.find("span", class_="onsale").get_text(strip=True) if ps.find("span", class_="onsale") else None

            # categorie
            meta = ps.find('div', class_='product_meta')
            categorie = None
            if meta:
                posted_in = meta.find('span', class_='posted_in')
                if posted_in and posted_in.find('a'):
                    categorie = posted_in.find('a').get_text(strip=True)

            # sous categorie
            bread = ps.find("nav", class_="woocommerce-breadcrumb")
            sous_categorie = None
            if bread:
                links = bread.find_all("a")
                if len(links) > 0:
                    # on prend le dernier lien d'ariane
                    # on verife que ce n'est pas Accueil
                    dernier_lien = links[-1].get_text(strip=True)
                    if dernier_lien.lower() not in ['accueil', 'home', 'boutique', 'shop']:
                        sous_categorie = dernier_lien


            description = ps.find("div", class_="electro-description").get_text(strip=True) if ps.find("div", class_="electro-description") else None

            # images
            imgs = ps.select(".woocommerce-product-gallery__wrapper img")
            img_files = []
            seen = set()
            for i, img in enumerate(imgs):
                src = img.get('src')
                if not src or src in seen: continue
                seen.add(src)

                # encodage url
                p_url = urlparse(urljoin(product_url, src))
                enc_url = f"{p_url.scheme}://{p_url.netloc}{quote(p_url.path)}"

                fname = os.path.basename(p_url.path).replace('\u2011', '-')
                fname = fname.encode('ascii', 'ignore').decode('ascii')

                try:
                    r_img = requests.get(enc_url, timeout=10)
                    with open(os.path.join(DOSSIER_IMG, fname), 'wb') as f:
                        f.write(r_img.content)
                    img_files.append(fname)
                except: continue

            nouveau_produit = {
                "nom": nom,
                "url": product_url,
                "prix_avant": prix_avant_reduction,
                "prix_apres": prix_apres_reduction,
                "reduction": reduction,
                "categorie": categorie,
                "sous_categorie": sous_categorie,
                "description": description,
                "images": img_files, # on met une liste
                "date_scraping": time.strftime("%d-%m-%Y")
            }

            produits_col.insert_one(nouveau_produit)
            print(f"Sauvegardé dans MongoDB : {nom}")

        except Exception as e:
            print(f"Erreur produit : {e}")
            time.sleep(5)

    page += 1


==> Page 1 --> https://nowtechcenter.com/boutique/page/1/
Nombre de produits trouvés sur cette page : 38
Déjà en base : https://nowtechcenter.com/produit/talkie-walkie-a522-portable-paire-de-2-avec-chargeur-bon-prix-en-vente-au-cameroun/
Déjà en base : https://nowtechcenter.com/produit/talkie-walkie-longue-portee-bf-uv25-en-vente-au-cameroun-bon-prix/
Déjà en base : https://nowtechcenter.com/produit/carte-de-ville-avec-notification-de-position-intelligent-bon-prix-en-vente-au-cameroun/
Déjà en base : https://nowtechcenter.com/produit/lecteur-dempreinte-hikvision-ds-k1201amf-bon-prix-en-vente/
Déjà en base : https://nowtechcenter.com/produit/access-point-ubiquiti-u7-outdoor-bon-prix-en-vente/
Déjà en base : https://nowtechcenter.com/produit/imprimante-primacy-2-lcd-duplex-expert-contactless-pm2-0019-bon-prix-en-vente/
Déjà en base : https://nowtechcenter.com/produit/switch-tp-link-tl-sg2424p-24-ports-giga-bon-prix-en-vente/
Déjà en base : https://nowtechcenter.com/produit/camera-ip-dah

In [ ]:
import pandas as pd
# On recup tous les docs de la collection
data = list(produits_col.find())
df = pd.DataFrame(data)
# On supprime la colonne '_id' generee par MongoDB 
if '_id' in df.columns:
    df = df.drop(columns=['_id'])

df.to_excel("newtech_mongodb_final.xlsx", index=False)
print("\nFichier Excel généré depuis MongoDB !")


Fichier Excel généré depuis MongoDB !


In [ ]:
import os
import shutil

DOSSIER_IMG = 'ImagesTech'

if os.path.exists(DOSSIER_IMG):
    
    shutil.make_archive(DOSSIER_IMG, 'zip', DOSSIER_IMG)
    print(f"Le dossier '{DOSSIER_IMG}' a été compressé en '{DOSSIER_IMG}.zip'")
else:
    print(f"Le dossier '{DOSSIER_IMG}' n'existe pas.")

Le dossier 'ImagesTech' a été compressé en 'ImagesTech.zip'
